# importer les packages

In [1]:
import numpy as np
import os  
import tensorflow as tf
from glob import glob

from keras.applications.vgg16 import VGG16 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from keras.models import Model 


# Importer la base de données

In [2]:
data_path = r'C:\Users\USER\Downloads\archive\chest_xray'

In [3]:
# specifier le chemin de la base de données train et test
train_dir=os.path.join(data_path,'train')
test_dir=os.path.join(data_path,'test')

In [4]:
# preparer le modele de base
IMG_SHAPE=(224,224,3)
base_model=VGG16(input_shape=IMG_SHAPE,include_top=False, weights='imagenet')
base_model.summary()



Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [5]:
# bloquer le modèle de base
base_model.trainable=False

In [7]:
# Ajouter les couches de sorties
x= Flatten()(base_model.output)
prediction=Dense(2,activation='softmax')(x)

In [8]:
# creation du modele global
model=Model(inputs=base_model.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
# compiler le modele
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# creer les photos augmentées
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

trainning_set=train_datagen.flow_from_directory(train_dir, target_size=(224,224),batch_size=32,class_mode='categorical')
test_set=test_datagen.flow_from_directory(test_dir, target_size=(224,224),batch_size=32,class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# appliquer le modele
model.fit_generator(trainning_set,epochs=5,validation_data=test_set)

C:\Users\USER\AppData\Local\Temp\ipykernel_21948\4159642344.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(trainning_set,epochs=5,validation_data=test_set)


Epoch 1/5


163/163 [==============================] - 640s 4s/step - loss: 0.2018 - accuracy: 0.9176 - val_loss: 0.2217 - val_accuracy: 0.9215
Epoch 2/5
133/163 [=======================>......] - ETA: 1:51 - loss: 0.1118 - accuracy: 0.9577

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(test_set)

In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))